Hi you, 

So here we are going to show a small demo of ctypeslib 2.0.

The purpose of ctypeslib is to transform C headers into ABI-compatible python code using ctypes.

First, lets start with a few easy C headers that we want to "convert" to python ctypes code.

In [1]:
c_headers = '''
/** very simple tests */
int version = 2;
char text[] = "Hello World!";
int minors[] = {2,0,0};
'''

we now can use the ctypeslib parser.

Note: the ctypeslib python library require the LLVM Clang library and its python bindings.

In [2]:
import ctypeslib
from ctypeslib.codegen import clangparser
# let's create a parser
parser = clangparser.Clang_Parser(flags=[])
# parse the c headers
parser.parse_string(c_headers)
# get the result
items = parser.get_result()
print items

[Variable(comment=None,init=Hello World!,typ=ArrayType(...),name=text,location=('/tmp/tmpQSo4jd.h', 4)), Variable(comment=very simple tests,init=2,typ=FundamentalType(...),name=version,location=('/tmp/tmpQSo4jd.h', 3)), Variable(comment=None,init=[2, 0, 0],typ=ArrayType(...),name=minors,location=('/tmp/tmpQSo4jd.h', 5))]


Now, we have parsed the C headers.

The parser returns an collection of intermediate python objects.

We want to produce a python source code out of it. Let's use the codegenerator.

In [3]:
# we need a output stream to store the generated code 
from StringIO import StringIO
output = StringIO()

from ctypeslib.codegen import codegenerator
# get the generator 
gen = codegenerator.Generator(output)
# and produce the code
gen.generate(parser, items)

The code is generated in the output stream (which could be a file)

In [4]:
print output.getvalue()

# -*- coding: utf-8 -*-
#
# TARGET arch is: []
# WORD_SIZE is: 8
# POINTER_SIZE is: 8
# LONGDOUBLE_SIZE is: 16
#
import ctypes




version = 2 # Variable ctypes.c_int32
text = 'Hello World!' # Variable ctypes.c_char * 12
minors = [2, 0, 0] # Variable ctypes.c_int32 * 3
__all__ = ['text', 'version', 'minors']



Let's try to load that code in the python console

In [5]:
# -*- coding: utf-8 -*-
#
# TARGET arch is: []
# WORD_SIZE is: 8
# POINTER_SIZE is: 8
# LONGDOUBLE_SIZE is: 16
#
import ctypes




version = 2 # Variable ctypes.c_int32
text = 'Hello World!' # Variable ctypes.c_char * 12
minors = [2, 0, 0] # Variable ctypes.c_int32 * 3
__all__ = ['text', 'version', 'minors']

Are the C variables now accessible in Python ?

In [6]:
print text
print 'Brought to you by ctypeslib version', version
print minors

Hello World!
Brought to you by ctypeslib version 2
[2, 0, 0]


Success !!
Let's try something harder.

In [7]:
structs="""
struct Name
{
  short member1;
  int member2;
  unsigned int member3;
  unsigned int member4;
  unsigned int member5;
} __attribute__((packed));

struct Name2
{
  short member1;
  int member2;
  unsigned int member3;
  unsigned int member4;
  unsigned int member5;
};

struct Node {
  unsigned int val1;
  void * ptr2;
  int * ptr3;
  struct Node2 * ptr4;
};

struct Node2 {
  unsigned char m1;
  struct Node * m2;
};

struct Node3 {
  struct Node * ptr1;
  unsigned char * ptr2;
  unsigned short * ptr3;
  unsigned int * ptr4;
  unsigned long * ptr5;
  unsigned long long * ptr6;
  double * ptr7;
  long double * ptr8;
  void * ptr9;
};"""

parser = clangparser.Clang_Parser(flags=[])
parser.parse_string(structs)
items = parser.get_result()
output = StringIO()
gen = codegenerator.Generator(output)
gen.generate(parser, items)

print output.getvalue()

# -*- coding: utf-8 -*-
#
# TARGET arch is: []
# WORD_SIZE is: 8
# POINTER_SIZE is: 8
# LONGDOUBLE_SIZE is: 16
#
import ctypes


# if local wordsize is same as target, keep ctypes pointer function.
if ctypes.sizeof(ctypes.c_void_p) == 8:
    POINTER_T = ctypes.POINTER
else:
    # required to access _ctypes
    import _ctypes
    # Emulate a pointer class using the approriate c_int32/c_int64 type
    # The new class should have :
    # ['__module__', 'from_param', '_type_', '__dict__', '__weakref__', '__doc__']
    # but the class should be submitted to a unique instance for each base type
    # to that if A == B, POINTER_T(A) == POINTER_T(B)
    ctypes._pointer_t_type_cache = {}
    def POINTER_T(pointee):
        # a pointer should have the same length as LONG
        fake_ptr_base_type = ctypes.c_uint64 
        # specific case for c_void_p
        if pointee is None: # VOID pointer type. c_void_p.
            pointee = type(None) # ctypes.c_void_p # ctypes.c_ulong
            clsnam

In [8]:
# lets load this output into a Python namespace
namespace = {}
exec output.getvalue() in namespace
namespace = ctypeslib.codegen.util.ADict(namespace)

import ctypes
print "sizeof(struct Name) ==", ctypes.sizeof(namespace.struct_Name)
print "sizeof(struct Name2) ==", ctypes.sizeof(namespace.struct_Name2)

sizeof(struct Name) == 18
sizeof(struct Name2) == 20


Un-surprisingly, the packed structure is smaller than the non-packed one.

In [1]:
n1 = namespace.struct_Node()
n2 = namespace.struct_Node2()
n3 = namespace.struct_Node3()

ptr_n2 = ctypes.pointer(n2)
n1.val1 = 42
n1.ptr4 = ptr_n2

n2.m1 = 43
n2.m2 = ctypes.pointer(n1)
n3.ptr1 = n2.m2

print "node3", n3
print "node3.ptr1", n3.ptr1
print "node3->ptr1.val1",n3.ptr1.contents.val1
print "node3->ptr1.ptr4->m1",n3.ptr1.contents.ptr4.contents.m1
print "node2.m1",n3.ptr1.contents.ptr4.contents.m1

NameError: name 'namespace' is not defined

If you want to use standard headers, you need to specify include path

In [7]:
!cat test/data/test-stdbool.c

#include <stdbool.h>

typedef struct s_foo {
	  bool bar1;
	    bool bar2;
	      bool bar3;
} foo;


usage: clang2py [-h] [-c] [-d] [--debug] [-e] [-k TYPEKIND] [-i] [-l DLL]
                [-m module] [-o OUTPUT] [-p DLL] [-q] [-r EXPRESSION]
                [-s SYMBOL] [-t TARGET] [-v] [-V] [-w W] [-x]
                [--show-ids SHOWIDS] [--max-depth N] [--clang-args CLANG_ARGS]
                files [files ...]
clang2py: error: argument files: can't open '../../test/data/test-stdbook.c': [Errno 2] No such file or directory: '../../test/data/test-stdbook.c'
